<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/1_process_exist_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 处理数据集 Part 1

* 数据集是根据trait和type爬取，所以数据会有冗余，需要合并
* 将合并的数据，更新到movie表中

#### 不需要每次离线构建模型都执行

In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce

In [0]:
df = pd.read_table('C:\T2\AI\douban\movie.csv',sep=',',encoding='utf-8')

In [0]:
df.head()

,类型,主演,地区,导演,特色,评分,电影名
0,剧情,徐峥|王传君|周一围|谭卓|章宇,中国大陆,文牧野,经典,8.9,我不是药神
1,剧情,冯小刚|许晴|张涵予|刘桦|李易峰,中国大陆,管虎,经典,7.8,老炮儿
2,剧情,王宝强|刘昊然|肖央|刘承羽|尚语贤,中国大陆,陈思诚,经典,6.7,唐人街探案2
3,剧情,任素汐|大力|刘帅良|裴魁山|阿如那,中国大陆,周申|刘露,经典,8.3,驴得水
4,剧情,徐峥|王宝强|李曼|李小璐|左小青,中国大陆,叶伟民,经典,7.5,人在囧途


In [0]:
%config ZMQInteractiveShell.ast_node_interactivity = 'all'
#df['主演'].apply(lambda x:x.split('|'))

In [0]:
# help(pd.core.series.Series)
movieinfo = df[df['电影名'] == '老炮儿']
types = movieinfo['类型'].tolist()
print(list(set(types)))

traits = movieinfo['特色'].tolist()
print(list(set(traits)))

print(movieinfo['主演'].iloc[0])


['剧情', '犯罪', '动作']
['感人', '经典']
冯小刚|许晴|张涵予|刘桦|李易峰


In [0]:
connection=pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                           user='noone',
                           password='Huawei12#$',
                           db='recsys',
                           port=3306,
                           charset ='utf8',
                           use_unicode=True)

# 更新电影数据
def update_one_movie_info(type_, actors, region, director, trait, rat, id_):
    try:
        sql = 'update movie set type=\'%s\',actors=\'%s\',region=\'%s\',director=\'%s\',trait=\'%s\',rat=\'%s\',enable=1 where id=\'%s\'' % (type_, actors, region, director, trait, rat, id_)
        #print(sql)
        with connection.cursor() as cursor:
            cout=cursor.execute(sql)
        connection.commit()
    except:
        connection.close()
        
# 逻辑停用一个电影数据
def disable_movie(id_):
    #print('id'+id_)
    try:
        sql = 'update movie set enable=0 where id=%s' % id_
        with connection.cursor() as cursor:
            cout=cursor.execute(sql)
        connection.commit()
    except:
        connection.close()

# 处理主函数，负责将多个冗余数据合并为一条电影数据，将地区，导演，主演，类型，特色等维度数据合并
def mainfunc():
    try:
        unable_list = []
        with connection.cursor() as cursor:
            sql='select id,name from movie'
            cout=cursor.execute(sql)
            print("数量： "+str(cout))

            for row in cursor.fetchall():
                #print(row[1])
                movieinfo = df[df['电影名'] == row[1]]
                if movieinfo.shape[0] == 0:
                    disable_movie(row[0])
                    print('disable movie ' + str(row[1]))
                else:
                    g = lambda x:movieinfo[x].iloc[0]
                    types = movieinfo['类型'].tolist()
                    types = reduce(lambda x,y:x+'|'+y,list(set(types)))
                    traits = movieinfo['特色'].tolist()
                    traits = reduce(lambda x,y:x+'|'+y,list(set(traits)))
                    update_one_movie_info(type_=types, actors=g('主演'), region=g('地区'), director=g('导演'), trait=traits, rat=g('评分'), id_=row[0])
                          
        connection.commit()

    finally:
        connection.close()
        
mainfunc()

数量： 9370
(4, 7)
(8, 7)
disable movie 剧场版魔法少女小圆 前篇 起始的故事
(4, 7)
disable movie 毒战
(18, 7)
(4, 7)
(4, 7)
disable movie 迷失天堂
disable movie 思维空间
(6, 7)
(13, 7)
(1, 7)
(8, 7)
(6, 7)
(2, 7)
(11, 7)
disable movie 阴阳相成
(9, 7)
disable movie 人类灭亡报告书
(4, 7)
(8, 7)
(1, 7)
(9, 7)
(4, 7)
(6, 7)
(16, 7)
(8, 7)
(5, 7)
(9, 7)
disable movie 服从
(4, 7)
(7, 7)
(10, 7)
disable movie 亚当与狗
(12, 7)
(3, 7)
disable movie 人类清除计划
(12, 7)
(12, 7)
disable movie 触摸恶魔
(5, 7)
(10, 7)
disable movie 重返大海
(5, 7)
disable movie 犀利人妻
(5, 7)
(4, 7)
disable movie 白金数据
disable movie 黑金杀机
disable movie 推理要在晚餐后SP
(2, 7)
(24, 7)
(4, 7)
disable movie 鬼打墙
(4, 7)
disable movie 致命录像带
(9, 7)
(12, 7)
(18, 7)
disable movie 海豹六队：突袭奥萨马本拉登
(4, 7)
(4, 7)
(4, 7)
(2, 7)
(6, 7)
(4, 7)
(12, 7)
(5, 7)
(6, 7)
(7, 7)
disable movie 天空之眼
(4, 7)
(2, 7)
(5, 7)
disable movie 怒火保镖
(4, 7)
(5, 7)
(3, 7)
(4, 7)
(5, 7)
disable movie 我的P.S.搭档
(5, 7)
(1, 7)
(4, 7)
disable movie 青魇
(8, 7)
disable movie 切尔诺贝利日记
disable movie 漫长的丝绸之路铁道～横跨中国2600公里之旅～
disable movie 

C:\Users\rexen\Anaconda2\envs\ml3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'director' at row 1")
  result = self._query(query)


disable movie 制服
disable movie 人间蒸发
(1, 7)
(20, 7)
(4, 7)
(12, 7)
disable movie 盲流感
disable movie 末日侵袭
disable movie 拆弹部队
(7, 7)
(4, 7)
(4, 7)
(5, 7)
(8, 7)
(5, 7)
(19, 7)
(1, 7)
disable movie 阁楼
(4, 7)
disable movie 森冤
disable movie 吓死鬼
disable movie 刺猬背西瓜
disable movie 狐狸送葡萄
disable movie 小蛋壳
(1, 7)
(6, 7)
(10, 7)
(10, 7)
(2, 7)
disable movie 美食、祈祷和恋爱
(2, 7)
(2, 7)
(3, 7)
(3, 7)
(1, 7)
disable movie 天才嘉年华
disable movie 2077日本锁国
(7, 7)
(6, 7)
(6, 7)
(3, 7)
(6, 7)
disable movie 维多利亚的秘密2012时装秀
(12, 7)
(11, 7)
(8, 7)
(4, 7)
disable movie 宫锁沉香
disable movie 阿帕鲁萨镇
disable movie 使命召唤
(4, 7)
(4, 7)
disable movie 彼德与狼
(7, 7)
(2, 7)
disable movie 刺杀据点
disable movie 风暴
(6, 7)
(4, 7)
(6, 7)
(2, 7)
(4, 7)
(10, 7)
(2, 7)
disable movie 3096天
(18, 7)
(3, 7)
disable movie 万能鉴定士Q：蒙娜丽莎之瞳
(2, 7)
(3, 7)
disable movie 反恐疑云
(6, 7)
(1, 7)
(2, 7)
(9, 7)
disable movie 华颐：吞噬怪物的孩子
(5, 7)
disable movie 黑鲸
(2, 7)
(8, 7)
disable movie 中国革命：1911年至1949年
disable movie 自虐之诗
disable movie 律政英雄
(2, 7)
(4, 7)
(4, 7)
(2, 